Import libraries

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import joblib
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model


Load data, model, scaler, and build test set

In [ ]:
data = yf.download("^IXIC", start="2010-01-01", end="2020-03-01")
prices = data[['Close']].values

model = load_model("model_nasdaq_hybrid.keras")
scaler = joblib.load("scaler_nasdaq_hybrid.joblib")


window_size = 60
scaled_data = scaler.transform(prices)
train_size = int(len(scaled_data) * 0.8)
test_data = scaled_data[train_size - window_size:]

def create_test_set(dataset, time_step=60):
    x, y = [], []
    for i in range(time_step, len(dataset)):
        x.append(dataset[i-time_step:i, 0])
        y.append(dataset[i, 0])
    return np.array(x), np.array(y)

x_test, y_test = create_test_set(test_data, window_size)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)

Predict and calculate MAPE

In [ ]:
predictions = model.predict(x_test)
predictions_rescaled = scaler.inverse_transform(predictions)
actual_rescaled = scaler.inverse_transform(y_test.reshape(-1, 1))


mape = np.mean(np.abs((actual_rescaled - predictions_rescaled) / actual_rescaled)) * 100
print(f"\n--- NASDAQ (^IXIC) RESULTS ---")
print(f"Paper Result: 3.34% | Your Result: {mape:.2f}%")

Plot actual vs predicted

In [ ]:
plt.figure(figsize=(14, 6))


plt.plot(actual_rescaled, color='black', label='Actual NASDAQ')
plt.plot(predictions_rescaled, color='purple', label=f'CNN-BiLSTM-Attn (MAPE: {mape:.2f}%)')


plt.title('NASDAQ Prediction - Overcoming 2020 Tech Volatility')
plt.xlabel('Days (Test Period starting from Jan 2020)') 
plt.ylabel('Stock Price (USD)')                          


plt.legend()
plt.grid(True, linestyle='--', alpha=0.7) 
plt.tight_layout()

plt.show()